
# Face recognition with Transfer Leraning

In [ ]:
import numpy as np

from keras.preprocessing import image

from keras.applications import vgg19

img_rows, img_cols=224,224

model=vgg19.VGG19(weights='imagenet', include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

In [ ]:
model.layers

In [ ]:
for (i,layer) in enumerate(model.layers):
    print(str(i)+"  "+model.layers[0].__class__.__name__+"  "+str(model.layers[i].trainable))

In [ ]:
model.layers[-1].output

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
for (i,layer) in enumerate(model.layers):
    print(str(i)+"  "+model.layers[0].__class__.__name__+"  "+str(model.layers[i].trainable))

In [ ]:
def addToModel(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addToModel(model, num_classes)

new_model = Model(inputs = model.input, outputs = FC_Head)

print(new_model.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data = r'\Transfer Learning\training'
validation_data = r'\Transfer Learning\validation'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 16
 
train_generator = train_datagen.flow_from_directory(
        train_data ,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data ,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("my_model.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [ checkpoint , earlystop]

# Note we use a very small learning rate 
new_model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 160
nb_validation_samples = 40
epochs = 2
batch_size = 16

history = new_model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks=callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

new_model.save("my_model.h5")

In [ ]:
train_generator.class_indices

In [ ]:
validation_generator.class_indices

### TESTING

In [ ]:
#### This is 1st method to test model:

from keras.models import load_model
classifier = load_model('my_model.h5')

import os
import cv2
from os import listdir
from os.path import isfile, join

person_dict = {"[0]": "Prajwal" ,"[1]":"Raj"}
person_dict_n = {"Prajwal": "Prajwal" , "Raj":"Raj"}

def test(name, pred, im):
    person = person_dict[str(pred)]
    b = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=b)
    cv2.putText(expanded_image, person, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getImage(path):
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + person_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)

for i in range(0,10):
    input_im = getImage('/Transfer Learning/test/')
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.25, fy=0.25, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    print(res)
    # Show image with predicted class
    #test("Prediction", res, input_original) 
    #cv2.waitKey(0)


#cv2.destroyAllWindows()

In [ ]:
#### This is 2nd method to test model:

from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.models import load_model
import numpy as np

modelnew=load_model("my_model.h5")

ori_img=image.load_img("/content/drive/My Drive/Colab Notebooks/test/3.jpg",target_size=(224,224))

img=image.img_to_array(ori_img)
import numpy as np

img=np.expand_dims(img,axis=0)

img=modelnew.predict(img)

if img[0][0]==1 and img[0][1]==0:
    
    prediction="Class 0"
elif img[0][0]==0 and img[0][1]==1:
    
    prediction="Class 1"
    
else:
    
    prediction="!! Unidentified !!"

print(prediction)

image.array_to_img(ori_img)